In [18]:
import numpy as np

Mol = 32e-3 # kg
Rg = 8.3144598  / Mol  # J / (mol * K) 

F_m = lambda vx, vy, vz, ux, uy, uz, T, n : n * ((1. / (2. * np.pi * Rg * T)) ** (3. / 2.)) * (
        np.exp(-((vx-ux)**2 + (vy-uy)**2 + (vz-uz)**2) / (2. * Rg * T))
)

def J(f, vmax, L):
    #fundamental constants
    Na = 6.02214129e+23
    kB = 1.381e-23 # J * K
    
    #gas parameters
    Mol = 32e-3 # kg
    Rg = 8.3144598  / Mol  # J / (mol * K) 
    m = 32e-3 / Na # kg
    
    Pr = 2. / 3.
    C = 127.
    T_0 = 292.25
    mu_0 = 20.18

    #vmax = np.sqrt(2 * Rg * T) * 4
    #L = 21
    
    h = 2. * vmax / L
    vx_ = np.linspace(-vmax+h/2, vmax-h/2, L)

    vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')

    assert np.all(vx[:,0,0] == vx_)
    assert np.all(vy[0,:,0] == vx_)
    assert np.all(vz[0,0,:] == vx_)

    n = (h ** 3) * np.sum(f)

    ux = (1. / n) * np.sum(vx * f)
    uy = (1. / n) * np.sum(vy * f)
    uz = (1. / n) * np.sum(vz * f)

    T = (2. / (3. * m * n * Rg)) * ((1. / 2.) * m * (h ** 3) * np.sum((vx*vx + vy*vy + vz*vz) * f)
                                    - (1. / 2.) * m * n * (ux*ux + uy*uy + uz*uz))

    Vx = vx - ux
    Vy = vy - uy
    Vz = vz - uz

    qx = (1. / 2.) * m * (h ** 3) * np.sum(Vx * (Vx*Vx + Vy*Vy + Vz*Vz) * f)
    qy = (1. / 2.) * m * (h ** 3) * np.sum(Vy * (Vx*Vx + Vy*Vy + Vz*Vz) * f)
    qz = (1. / 2.) * m * (h ** 3) * np.sum(Vz * (Vx*Vx + Vy*Vy + Vz*Vz) * f)

    rho = m * n

    p = rho * Rg * T

    (cx, cy, cz) = (Vx, Vy, Vz) / ((2. * Rg * T) ** (1. / 2.))

    Sx = (1. / n) * (h ** 3) * np.sum(cx * (cx*cx + cy*cy + cz*cz) * f)
    Sy = (1. / n) * (h ** 3) * np.sum(cy * (cx*cx + cy*cy + cz*cz) * f)
    Sz = (1. / n) * (h ** 3) * np.sum(cz * (cx*cx + cy*cy + cz*cz) * f)


    mu = mu_0 * ((T_0 + C) / (T + C)) * ((T / T_0) ** (3. / 2.))


    f_plus = F_m(vx, vy, vz, ux, uy, uz, T, n) * (1. + (4. / 5.) * (1. - Pr) * (cx*Sx + cy*Sy + cz*Sz) * 
                                      (cx*cx + cy*cy + cz*cz - (5. / 2.)))

    J = (p / mu) * (f_plus - f)

    return J

In [19]:
vmax = 1000.
L = 20
    
h = 2. * vmax / L
vx_ = np.linspace(-vmax+h/2, vmax-h/2, L)

print vx_

vx, vy, vz = np.meshgrid(vx_, vx_, vx_, indexing='ij')

f = F_m(vx, vy, vz, 0, 0, 0, 300, 1.88191915312e+25)

print J(f, vmax, L)

[-950. -850. -750. -650. -550. -450. -350. -250. -150.  -50.   50.  150.
  250.  350.  450.  550.  650.  750.  850.  950.]
1.8801290832462725e+25
[[[-4.11461294e+11 -1.21485712e+12 -3.16449321e+12 ... -3.16449321e+12
   -1.21485712e+12 -4.11461294e+11]
  [-1.21485712e+12 -3.56547153e+12 -9.22998910e+12 ... -9.22998910e+12
   -3.56547153e+12 -1.21485712e+12]
  [-3.16449321e+12 -9.22998910e+12 -2.37389918e+13 ... -2.37389918e+13
   -9.22998910e+12 -3.16449321e+12]
  ...
  [-3.16449321e+12 -9.22998910e+12 -2.37389918e+13 ... -2.37389918e+13
   -9.22998910e+12 -3.16449321e+12]
  [-1.21485712e+12 -3.56547153e+12 -9.22998910e+12 ... -9.22998910e+12
   -3.56547153e+12 -1.21485712e+12]
  [-4.11461294e+11 -1.21485712e+12 -3.16449321e+12 ... -3.16449321e+12
   -1.21485712e+12 -4.11461294e+11]]

 [[-1.21485712e+12 -3.56547153e+12 -9.22998910e+12 ... -9.22998910e+12
   -3.56547153e+12 -1.21485712e+12]
  [-3.56547153e+12 -1.03907672e+13 -2.67006654e+13 ... -2.67006654e+13
   -1.03907672e+13 -3.5654